# A demo how to work with `QAOAParameter` classes

Farhi et al proposed in their original paper about QAOA the following Ansatz for the QAOA circuit:

$$
U = e^{-i \beta_p H_0} e^{-i \gamma_p H_1} \cdots e^{-i \beta_0 H_0} e^{-i \gamma  H_1}.
$$
...

### ToDo for this Demo
 - [ ] Finish the explanation of the different parametrizations
 - [ ] Explain _well_ what the `gammas_pairs`, `gammas_singles` and `betas` are in the raw format.
 - [ ] Demo _all_ different parametrizations that we have

In [1]:
# import the standard modules from python
import numpy as np
import matplotlib.pyplot as plt

# import the neccesary pyquil modules
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters that we want to demo
from qaoa.parameters import AdiabaticTimestepsQAOAParameters, AlternatingOperatorsQAOAParameters

Let us first create a hamiltonian to work with:

In [2]:
# create a hamiltonian on 3 qubits with 2 coupling terms and 1 bias term
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0


## Building the parameters automatically with `.from_hamiltonian()`

The easiest way to create `AlternatingOperatorsQAOAParameters` that work exactly with our Hamiltonian, is to use the `AlternatingOperatorsQAOAParameters.from_hamiltonian` function:

In [8]:
timesteps = 2       # equivalent to p in Farhis original paper
time = 1            # total time of the annelaing schedule
params = AlternatingOperatorsQAOAParameters.from_hamiltonian(hamiltonian,
                                                           timesteps,
                                                           time = time) 
print(params)

register: [0, 1, 2]
betas: [0.375, 0.125]
qubits_singles: [0]
gammas_singles: [0.125, 0.375]
qubits_pairs: [[0, 1], [0, 2]]
gammas_pairs: [0.125, 0.375]



We see, that if we print `params`, it gives us the timesteps the angles and on which combinations of qubits they act. But we can also have a look at the _internal_ `gammas_singles`, `gammas_pairs` and `betas` that were automatically calculated under the hood. These are the coefficients that are applied to the single terms in the exponentiated hamiltonian.

In [9]:
print("\n betas:\n", params.betas)
print("\n gammas_singles:\n", params.gammas_singles)
print("\n gammas_pairs:\n", params.gammas_pairs)


 betas:
 [[0.375, 0.375, 0.375], [0.125, 0.125, 0.125]]

 gammas_singles:
 [[-0.0625], [-0.1875]]

 gammas_pairs:
 [[0.0875, 0.15], [0.26249999999999996, 0.44999999999999996]]


Now we could pass these `params` straight to a cost_function that we built with `qaoa.cost_function.QAOACostFunctionOnQVM` or with `qaoa.cost_function.QAOACostFunctionOnWFSim`. But often we will be interested in modifying the parameters, before running QAOA with them. The way the cost_function does this, is by getting a flat list with parameters via `params.raw()`, changing the values in that list and feeding it back to `params` via `params.update()`:

In [14]:
# get the raw parameters and print them
raw = params.raw()
print("the raw parameters:\n", raw)

# change a value and put it back via params.update()
print("\nthe new raw parameters:\n", raw)
raw[0] = 0.0
params.update(raw)
print("\nand the updated parameters:\n", params)

the raw parameters:
 [0.0, 0.125, 0.125, 0.375, 0.125, 0.375]

the new raw parameters:
 [0.0, 0.125, 0.125, 0.375, 0.125, 0.375]

and the updated parameters:
 register: [0, 1, 2]
betas: [0.0, 0.125]
qubits_singles: [0]
gammas_singles: [0.125, 0.375]
qubits_pairs: [[0, 1], [0, 2]]
gammas_pairs: [0.125, 0.375]



and we can convince us that under the hood the `gammas_singles`, `gammas_pairs` and `betas` were also automaticall updated:

In [15]:
print("\n betas:\n", params.betas)
print("\n gammas_singles:\n", params.gammas_singles)
print("\n gammas_pairs:\n", params.gammas_pairs)


 betas:
 [[0.0, 0.0, 0.0], [0.125, 0.125, 0.125]]

 gammas_singles:
 [[-0.0625], [-0.1875]]

 gammas_pairs:
 [[0.0875, 0.15], [0.26249999999999996, 0.44999999999999996]]


## Building the parameters manually

Sometimes we don't want to build a QAOAParametersObject automaticall via `.from_hamiltonian()`, but want some more finde graned control. So we can instead specify all that needs to be know manually and passing that to the constructor.
Fully specified QAOAParameters consist of constant parameters, that don't change over the lifetime of a QAOA optimization run and parameters that are varied during the optimization.

The constant parameters are the number of timesteps, the qubits that have bias terms and the qubits that are coupled as well as the cofficients of all these terms in the hamiltonian.

The parameters that are variable, are in the case of `AdiabaticTimestepsQAOAParameters`, the times at which to change the hamiltonian. In the case of `AlternatingOperatorsQAOAParameters` they are the `gammas_singles`, `gammas_pairs` and `betas`.

In [18]:
# the constant parameters
reg = [0, 1, 2]       # The register of _all_ qubits in the hamiltonian
                      # (or all the qubits, the X-rotations should act on) 
qubits_singles = [0]              # list of qubits with bias terms
qubits_pairs   = [[0, 1], [0, 2]] # list of qubit_pairs of coupled qubits
ntimesteps     = 3                # the number of timesteps

# put them into a nice tuple
constant_parameters = (reg, qubits_singles, qubits_pairs, ntimesteps, hamiltonian)


# the variable parameters --- in this case the timesteps
betas = [0.25, 0.5, 1.0]
gammas_singles = [1.0, 0.5, 0.3]
gammas_pairs = [1.0, 0.5, 0.3]
# and make them a tuple. Seems superflous in this case, but other
# parametrizations have more than one variable parameter.
variable_parameters = (betas, gammas_singles, gammas_pairs)
manual_params = AlternatingOperatorsQAOAParameters(constant_parameters,
                                                 variable_parameters)
print(manual_params)

register: [0, 1, 2]
betas: [0.25, 0.5, 1.0]
qubits_singles: [0]
gammas_singles: [1.0, 0.5, 0.3]
qubits_pairs: [[0, 1], [0, 2]]
gammas_pairs: [1.0, 0.5, 0.3]



Now if we want to change this parameters, we can do it again by getting the flat list of all of them with `manual_params.raw()`, changing them and putting them back with `manual_params.update()`. But this requires, that we know where in the long list of parameters the one is, that we want to change. So we can instead use `manual_params.update_variable_parameters` as follows:

In [19]:
betas = [1.0, 1.0, 1.0]
gammas_singles = [2.0, 2.0, 2.0]
gammas_pairs = [-.3, -.3, -.3]
variable_parameters = (betas, gammas_singles, gammas_pairs)
manual_params.update_variable_parameters(variable_parameters)

print(manual_params)

register: [0, 1, 2]
betas: [1.0, 1.0, 1.0]
qubits_singles: [0]
gammas_singles: [2.0, 2.0, 2.0]
qubits_pairs: [[0, 1], [0, 2]]
gammas_pairs: [-0.3, -0.3, -0.3]



We see, that the `betas`, `gammas_singles` and `gammas_pairs` have changed, while the constant parameters `register`, `qubits_singles` and `qubits_pairs` didn't change. We can also see, what happened to the final coefficients, that are passed to `qaoa._qaoa_annealing_program`:

In [20]:
print("\n betas:\n", params.betas)
print("\n gammas_singles:\n", params.gammas_singles)
print("\n gammas_pairs:\n", params.gammas_pairs)


 betas:
 [[0.0, 0.0, 0.0], [0.125, 0.125, 0.125]]

 gammas_singles:
 [[-0.0625], [-0.1875]]

 gammas_pairs:
 [[0.0875, 0.15], [0.26249999999999996, 0.44999999999999996]]
